# Wise INR → USD Transfer Route: 8-Week Launch Performance Review

## Executive Summary

Wise’s Regional Expansion Tribe aims to unlock fast and low-cost cross-border payments by launching new currency routes. Following initial validation that the INR → USD route had sufficient market demand, we analysed usage data from the first eight weeks post-launch to assess early performance and highlight areas requiring optimisation.

This review examines the launch through three main lenses:
1.	Post-launch adoption
2.	Funnel analysis
3.	Friction analysis

Together, these provide a holistic view of how successfully customers (both new and existing) across different regions are discovering, initiating, and completing transfers on the new route, as well as where early inefficiencies may be suppressing conversion. The goal is to identify actionable opportunities to stabilise and improve performance during the post-launch period.


## Overall Performance Assessment: 🟡 <u>Moderate Success</u>

The INR → USD route has successfully validated market demand but is underperforming on conversion and sustained growth. While initial adoption showed promise, significant friction points are preventing the route from reaching its full potential.

### Performance Scorecard

| Dimension | Observations | Status | Remarks |
|-----------|--------------|--------|------------|
| **Market Demand** | ✅ Strong | ✅ Good | Transfer Created volumes confirm latent demand exists |
| **W-o-W Growth** | +283% from week 1-2, then declining to negative | 🔴 Concern | Strong launch spike not sustained; growth hovered near 5%, then turned negative by Week 8 |
| **End-to-End Conversion** | ~24% | 🔴 Concern | Significant drop-off between Created → Transferred |
| **Regional Performance** | APAC growing strongest | ✅ Good | Primary market showing strongest trajectory |
| **Platform Parity** | 60%+ Android gap | 🔴 Critical | Android APAC conversion abyssmal |

**Verdict:** Route has validated demand but requires immediate intervention on conversion bottlenecks. Current trajectory will not deliver business case projections without product fixes.

### Priority Areas for Optimisation

#### 1. **P0: Android APAC — Critical Failure Point**

- **Issue:** 1.6% end-to-end conversion (vs. 31% on iOS) — worst performing segment
- **Root Cause:** Platform-specific UX friction in Android funding flow; only 1.8% convert from Created → Funded
- **Action:** UX deep-dive + user research on Android payment flow in APAC
- **Impact:** Close gap to potentially recover significant monthly transfers (targeting iOS parity of ~25% end-to-end conversion)

#### 2. **P1: New User Activation**

- **Issue:** 18% conversion vs. 32% for existing users — losing 80% of new users at funding stage
- **Root Cause:** LRS documentation complexity; unclear funding methods for first-time users
- **Action:** Simplify onboarding, progressive disclosure for compliance docs, contextual help
- **Impact:** Improved acquisition efficiency and sustainable growth

#### 3. **P2: European Completion Times**

- **Issue:** 2.5 days average (2× slower than APAC)
- **Root Cause:** Banking partner delays, compliance checks
- **Action:** Review partner SLAs, add faster payment options
- **Impact:** Improved NPS, increased repeat usage in the long-term

## 1. Setup & Data Overview

In [1]:
# Import libraries
import warnings
from pathlib import Path

import pandas as pd
import plotly
import plotly.express as px
import plotly.graph_objects as go
import yaml
from plotly.subplots import make_subplots

from src.database_manager import DatabaseManager

warnings.filterwarnings("ignore")

In [2]:
# Load YAML config
with open("../config/config.yaml", "r") as f:
    config = yaml.safe_load(f)

# Import config variables
db_path = Path("../") / config["database"]["database_path"]
data_path = Path("../") / config["database"]["raw_data_path"]
sql_path = Path("../") / config["database"]["table_schema_path"]
table_name = config["database"]["table_name"]

# Initialize database
with DatabaseManager(db_path) as db:
    db.initialise_database(
        csv_file=data_path,
        table_name=table_name,
        if_exists="replace",
        table_schema=sql_path,
    )

# Create database connection
db = DatabaseManager(db_path)

2025-12-09 22:17:59.266 | DEBUG    | src.database_manager:__init__:39 - DatabaseManager initialised with database path: ../database/wise_analytics.duckdb
2025-12-09 22:17:59.279 | DEBUG    | src.database_manager:connect:50 - Connected to database: ../database/wise_analytics.duckdb
2025-12-09 22:17:59.279 | INFO     | src.database_manager:initialise_database:131 - Initialising database: ../database/wise_analytics.duckdb
2025-12-09 22:17:59.280 | INFO     | src.database_manager:create_table_from_schema:158 - Creating table from schema: ../sql/create_transactions_table.sql
2025-12-09 22:17:59.281 | INFO     | src.database_manager:execute_sql_file:112 - Executing SQL file: ../sql/create_transactions_table.sql
2025-12-09 22:17:59.290 | DEBUG    | src.database_manager:execute_query:88 - Query executed successfully. Returned 0 rows.
2025-12-09 22:17:59.291 | INFO     | src.database_manager:initialise_database:140 - Loading data from: ../data/wise_funnel_events.csv
2025-12-09 22:17:59.329 | IN

To begin, we perform some basic exploratory data analysis (EDA) to validate the structure and quality of our dataset.

This includes checking for missing values, confirming the post-launch date range covered by the events, and reviewing the distribution of users across key dimensions such as region, platform, and experience type.

These preliminary checks help ensure the dataset is reliable for downstream analysis and provide initial context about the composition of users engaging with the new INR → USD route.

In [3]:
# Execute SQL query
df = db.execute_sql_file("../sql/general_eda.sql")

# Check post-launch period
min_event_date = df["EVENT_DATE"].min()
max_event_date = df["EVENT_DATE"].max()
print(f"Post-launch period: {min_event_date.strftime("%d %b %Y")} to {max_event_date.strftime("%d %b %Y")}\n")

# Check for nulls
print("Check for nulls:")
print(df.isnull().sum())

2025-12-09 22:17:59.475 | INFO     | src.database_manager:execute_sql_file:112 - Executing SQL file: ../sql/general_eda.sql
2025-12-09 22:17:59.482 | DEBUG    | src.database_manager:connect:50 - Connected to database: ../database/wise_analytics.duckdb
2025-12-09 22:17:59.505 | DEBUG    | src.database_manager:execute_query:88 - Query executed successfully. Returned 73440 rows.


Post-launch period: 01 Jan 2024 to 01 Mar 2024

Check for nulls:
EVENT_NAME    0
EVENT_DATE    0
USER_ID       0
REGION        0
PLATFORM      0
EXPERIENCE    0
dtype: int64


In [4]:
# Execute SQL query
df = db.execute_sql_file("../sql/eda_pivot_table.sql")

# Generate pivot table for quick overview on user counts
df.pivot_table(index=["REGION", "PLATFORM"], columns="EXPERIENCE", aggfunc="sum", values="USER_COUNT")

2025-12-09 22:17:59.525 | INFO     | src.database_manager:execute_sql_file:112 - Executing SQL file: ../sql/eda_pivot_table.sql
2025-12-09 22:17:59.538 | DEBUG    | src.database_manager:execute_query:88 - Query executed successfully. Returned 18 rows.


EXPERIENCE        Existing   New
REGION  PLATFORM                
APAC    Android       2903  3424
        Web           1452  1720
        iOS           2843  3513
Europe  Android       1556  6378
        Web            947  3856
        iOS            653  2575
NorthAm Android       1104   812
        Web           2257  1636
        iOS           2210  1619

## 2. Adoption Analysis

### 2.1 Overall Transfer Volume Trend

To assess early adoption of the INR → USD route, we track how users progress through the key stages of the Wise transfer lifecycle:
1. **Transfer Created**: The user initiates a transfer
2. **Transfer Funded**: The user successfully adds money
3. **Transfer Transferred**: The transfer is completed and delivered

These stages help us distinguish between initial demand (Created) and realised usage (Transferred), while also highlighting where friction may occur in between.

In [5]:
# Execute SQL query
df = db.execute_sql_file("../sql/funnel_stage_time_series.sql")

# Resample time series and aggregate to weekly counts
df.set_index("EVENT_DATE", inplace=True)
df = df.groupby("EVENT_NAME").resample("W").agg(COUNT=("COUNT", "sum")).reset_index()

# Plot time series
fig = px.line(
    df,
    x="EVENT_DATE",
    y="COUNT",
    title="Weekly Transfer Events by Stage (All Regions)",
    color="EVENT_NAME",
    markers=True,
)
fig.update_layout(
    width=1200,
    height=500,
    xaxis_title="Week",
    yaxis_title="Count",
    legend_title="Transfer Stage",
)
fig.show()

2025-12-09 22:17:59.641 | INFO     | src.database_manager:execute_sql_file:112 - Executing SQL file: ../sql/funnel_stage_time_series.sql
2025-12-09 22:17:59.648 | DEBUG    | src.database_manager:execute_query:88 - Query executed successfully. Returned 1079 rows.


**Observations:**
- At a high level, the sharp rise in Transfer Created during the first seven weeks demonstrates strong latent demand and successful early adoption of the INR → USD route. However, the gap between Created and Transferred widens over time, signalling growing conversion friction that prevents a meaningful share of users from completing their transfers.

- From a launch perspective, interest and initial take-up are strong, but the overall customer journey still requires optimisation. Improving the experience between creation and completion will be critical to translating this demand into actual transferred volume and achieving healthier end-to-end conversion rates.

- Between 25 Feb and 3 Mar, transfer volumes declined across all funnel stages. The fact that the drop is consistent from Created through Transferred suggests a broader shift in user activity rather than stage-specific friction. This pattern may point to seasonal effects or external factors temporarily reducing demand during this period, though further data would be needed to confirm seasonality.

### 2.2 Completed Transfers — The North Star Metric

Although each step reveals some insight about engagement and potential drop-offs, the most important metric for assessing launch performance is **Transfer Transferred**.

Completed transfers reflect actual revenue and directly impact the route’s P&L, making this the **primary KPI** for determining whether the route is gaining sustainable traction and where we should focus future optimisation efforts.

In [6]:
# Execute SQL query
df = db.execute_sql_file("../sql/funnel_stage_time_series.sql")

# Mask for completed transfers only
is_transferred_yes = df["EVENT_NAME"] == "Transfer Transferred"
df = df[is_transferred_yes]

# Resample time series and aggregate to weekly counts
df.set_index("EVENT_DATE", inplace=True)
df = df.resample("W").agg(COUNT=("COUNT", "sum")).reset_index()

# Plot time series
fig = px.line(
    df,
    x="EVENT_DATE",
    y="COUNT",
    title="Weekly Completed Transfers (Transfer Transferred)",
    markers=True,
)
fig.add_hline(
    y=df["COUNT"].mean(),
    line_dash="dash",
    line_color="red",
    annotation_text=f"Average: {df['COUNT'].mean():.0f}",
)
fig.update_layout(
    width=1200,
    height=500,
    xaxis_title="Week",
    yaxis_title="No. of Completed Transfers",
)
fig.show()

2025-12-09 22:18:00.607 | INFO     | src.database_manager:execute_sql_file:112 - Executing SQL file: ../sql/funnel_stage_time_series.sql
2025-12-09 22:18:00.611 | DEBUG    | src.database_manager:execute_query:88 - Query executed successfully. Returned 1079 rows.


**Observations:**
- Completed transfers increased significantly in the first three weeks but then began to level off, suggesting that sustained growth may be weaker than initial uptake implied. Although volumes from 21 Jan to 25 Feb remained above the overall eight-week average, the upward momentum tapered, indicating that early demand did not continue to scale at the same pace.

### 2.3 Week-over-Week Growth Analysis

To better understand the trajectory of adoption during the first eight weeks post-launch, we calculate week-over-week (W-o-W) growth for completed transfers, the primary indicator of actual usage.

This view makes it evident that growth has stagnated considerably after the initial launch spike. While early weeks showed positive momentum, subsequent periods failed to maintain the same pace, with several weeks showing flat or declining growth.

For a newly launched currency route, we would [typically aim for sustained 5% W-o-W growth to indicate healthy adoption](https://www.investopedia.com/terms/g/growthrates.asp#:~:text=What%20Is%20a%20Good%20Growth%20Rate%20for,exceptional%20growth%20rate%20is%2010%25%20per%20week.), increasing user intent, and long-term route viability. Falling short of this benchmark suggests that although the initial demand exists, the route is not yet converting or scaling at the level required to drive meaningful bottom-line impact for Wise.

In [7]:
# Execute SQL query
df = db.execute_sql_file("../sql/funnel_stage_time_series.sql")

# Mask for completed transfers only
is_transferred_yes = df["EVENT_NAME"] == "Transfer Transferred"
df = df[is_transferred_yes]

# Resample time series and aggregate to weekly counts
df.set_index("EVENT_DATE", inplace=True)
df = df.resample("W").agg(COUNT=("COUNT", "sum")).reset_index()

# Calculate W-o-W Growth
df["COUNT_LAG"] = df["COUNT"].shift(1)
df["WOW_GROWTH"] = (df["COUNT"] - df["COUNT_LAG"]) / df["COUNT_LAG"]

# Plot time series
fig = px.line(
    df,
    x="EVENT_DATE",
    y="WOW_GROWTH",
    title="Week-over-Week Growth Rate (All Events)",
    markers=True,
)
fig.add_hline(
    y=0.05,
    line_dash="dash",
    line_color="green",
    annotation_text="Target: 5%",
)
fig.update_layout(
    width=1200,
    height=500,
    xaxis_title="Week",
    yaxis_title="W-o-W Growth Rate",
    yaxis_tickformat=".0%",
)
fig.show()

2025-12-09 22:18:00.639 | INFO     | src.database_manager:execute_sql_file:112 - Executing SQL file: ../sql/funnel_stage_time_series.sql
2025-12-09 22:18:00.644 | DEBUG    | src.database_manager:execute_query:88 - Query executed successfully. Returned 1079 rows.


**Observation:**
- W-o-W growth spiked by 283% between Weeks 1 and 2, reflecting strong early adoption momentum.
- However, growth quickly tapered and hovered only marginally around the 5% target threshold in subsequent weeks before turning negative by Week 8.
- This pattern suggests that while initial interest was high, users were not consistently progressing to completed transfers, indicating potential friction points that are limiting sustained adoption.

### 2.4 Regional Adoption by Transfer Stages

When evaluating a newly launched currency route, it is important not only to understand overall adoption, but also how engagement varies across different regions. Each market has its own levels of latent and expressed demand, as well as unique behavioural patterns that influence funnel progression.

To explore this, we plot the transfer lifecycle stages by region, allowing us to detrermine which regions show strong interest but weaker follow-through, where conversion is healthy, and where potential gaps or opportunities exist.

Identifying these regional discrepancies enables Wise to refine targeting, prioritise operational improvements, and tailor interventions to markets with the greatest potential impact.

In [8]:
# Execute SQL query
df = db.execute_sql_file("../sql/funnel_stage_time_series.sql")

# Resample time series and aggregate to weekly counts
df.set_index("EVENT_DATE", inplace=True)
df = (
    df.groupby(["EVENT_NAME", "REGION"])
    .resample("W")
    .agg(COUNT=("COUNT", "sum"))
    .reset_index()
)

# Plot time series
fig = px.line(
    df,
    x="EVENT_DATE",
    y="COUNT",
    facet_col="EVENT_NAME",
    color="REGION",
    title="Transfer Events by Region and Stage",
    markers=True,
)
fig.update_layout(
    width=1200,
    height=500,
    xaxis_title="Week",
    yaxis_title="Event Count",
    legend_title_text="Region",
)
fig.update_xaxes(title_text="Week")
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))

fig.show()

2025-12-09 22:18:00.673 | INFO     | src.database_manager:execute_sql_file:112 - Executing SQL file: ../sql/funnel_stage_time_series.sql
2025-12-09 22:18:00.677 | DEBUG    | src.database_manager:execute_query:88 - Query executed successfully. Returned 1079 rows.


**Observations:**
- Interestingly, early latent demand (Transfer Created) was strongest in Europe and North America, with APAC initially lagging behind. However, APAC demand accelerated sharply in the second month, suggesting that early awareness or onboarding friction may have slowed initial adoption in the home region.
- Despite this late surge in intent, APAC’s demand did not fully translate into completed transfers. Between 4 Feb and 1 Mar, the number of completed transfers in APAC was only marginally higher (at most around 70% more) than in Europe and North America. This indicates that although APAC users eventually showed strong interest, they faced substantial friction converting that intent into successful transfers.

### 2.5 Completed Transfers by Region & Experience

To better understand adoption patterns, we break down completed transfers by region and user experience type (new vs. existing users). This reveals how different user segments contribute to early usage of the INR → USD route.

In [9]:
# Execute SQL query
df = db.execute_sql_file("../sql/funnel_stage_time_series.sql")

# Mask for completed transfers only
is_transferred_yes = df["EVENT_NAME"] == "Transfer Transferred"
df = df[is_transferred_yes]

# Ensure proper datetime
df["EVENT_DATE"] = pd.to_datetime(df["EVENT_DATE"])
df.set_index("EVENT_DATE", inplace=True)

# Group by both Region & Experience before resampling
df_weekly = (
    df.groupby(["REGION", "EXPERIENCE"])
    .resample("W")
    .agg(COUNT=("COUNT", "sum"))
    .reset_index()
)

# Plot time series
fig = px.line(
    df_weekly,
    x="EVENT_DATE",
    y="COUNT",
    color="REGION",
    facet_col="EXPERIENCE",
    title="Weekly Completed Transfers: Region by User Experience",
    markers=True,
)
fig.update_layout(
    width=1200,
    height=500,
    xaxis_title="Week",
    yaxis_title="Event Count",
    legend_title_text="Region",
)
fig.update_xaxes(title_text="Week")
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1] + " Users"))

fig.show()

2025-12-09 22:18:00.733 | INFO     | src.database_manager:execute_sql_file:112 - Executing SQL file: ../sql/funnel_stage_time_series.sql
2025-12-09 22:18:00.737 | DEBUG    | src.database_manager:execute_query:88 - Query executed successfully. Returned 1079 rows.


**Observations:**
- Across APAC, the volume of completed transfers is consistently higher among existing users compared to new users. This suggests that users already familiar with Wise are more comfortable completing transfers on the new route, while new users may still face onboarding or trust-related friction.

- A similar, although less pronounced, pattern appears in North America, where existing users also contribute more steadily than new users.

- In contrast, Europe shows the opposite trend where new users contribute a significantly larger share of completed transfers than existing users, especially in the first month. This may indicate stronger acquisition momentum in Europe, where new customers are discovering and adopting the route quickly, even if existing user engagement is more modest.

- These regional differences highlight how adoption dynamics vary by market and suggest that the product, growth, and activation strategies may need to be tailored differently for APAC, Europe, and North America.


## 3. Funnel Analysis (Diagnostic Deep-Dive)

While adoption analysis shows *what* happened, funnel analysis explains *why* by identifying drop-off points and conversion barriers.

In this section, we explore in detail the different customer segments so that we isolate different customer journies in greater detail.

### 3.1 Overall Funnel by Region & Experience
In this section, we analyse the customer journey by comparing funnel performance across new and existing users.

Breaking down the transfer lifecyle by user experience type helps us to identify where users are dropping off and whether onboarding or app familiarity are creating friction, particularly for first-time users.

This view allows us to assess whether the new route is resonating equally across segments or if specific user groups require targeted improvements to increase successful transfer completion.

In [10]:
# Execute SQL query
df = db.execute_sql_file("../sql/funnel_by_region_experience.sql")

# Get unique regions
regions = df["REGION"].unique()
experiences = sorted(df["EXPERIENCE"].unique())

# Define colors for experiences
color_map = {"Existing": "#636efa", "New": "#EF553B"}

# Create subplots
fig = make_subplots(
    rows=1,
    cols=len(regions),
    column_titles=list(regions),
    shared_yaxes=True,
    vertical_spacing=0.05,
)

# Add funnel traces for each region and experience
for col, region in enumerate(regions, start=1):
    # Loop through each experience type
    for experience in experiences:
        # Filter data for this region AND experience
        mask = (df["REGION"] == region) & (df["EXPERIENCE"] == experience)
        df_filtered = df[mask].sort_values("USER_COUNT", ascending=False)

        # Add funnel trace
        fig.add_trace(
            go.Funnel(
                name=experience,
                y=df_filtered["EVENT_NAME"],
                x=df_filtered["USER_COUNT"],
                textinfo="value+percent previous",
                textposition="inside",
                marker=dict(color=color_map[experience]),  # Set color here
                customdata=df_filtered[
                    ["CONVERSION_RATE", "DROP_OFF_RATE", "CUMULATIVE_CONVERSION_RATE"]
                ].values,
                hovertemplate="<b>%{y}</b><br>"
                + "Experience: <b>" + experience + "</b><br><br>"
                + "Users: <b>%{x:,}</b><br>"
                + "Conversion: <b>%{customdata[0]:.1%}</b><br>"
                + "Drop-off: <b>%{customdata[1]:.1%}</b><br>"
                + "Cumulative: <b>%{customdata[2]:.1%}</b><br>"
                + "<extra></extra>",
                showlegend=(col == 1),  # Show legend only for first subplot
            ),
            row=1,
            col=col,
        )

# Update layout
fig.update_layout(
    width=1200,
    height=500,
    showlegend=True,
    title_text="Conversion Funnel by Region and Experience",
    yaxis_title_text="Funnel Stage",
    legend_title_text="User Experience",
    template="plotly_white",
)

fig.show()

2025-12-09 22:18:00.805 | INFO     | src.database_manager:execute_sql_file:112 - Executing SQL file: ../sql/funnel_by_region_experience.sql
2025-12-09 22:18:00.826 | DEBUG    | src.database_manager:execute_query:88 - Query executed successfully. Returned 18 rows.


**Observations:**
- New users consistently demonstrate much lower conversion rates than existing users across all regions, with the steepest drop occurring between Transfer Created → Transfer Funded. This indicates that early-stage friction—such as onboarding, account verification, or funding setup—is disproportionately affecting first-time customers.

- While both experience types show expected funnel drop-off, the decline is significantly more pronounced for new users, especially in APAC, where fewer than one in five new users move from creation to funding. This pattern suggests that new customers may encounter more complexity or uncertainty in the early steps of the process, leading to higher abandonment before reaching transfer completion.

- Across most regions (Europe and North America) roughly half of users fail to complete the journey from funding to transfer completion, indicating a significant opportunity to optimise mid- and late-stage conversion. Improving this step could directly strengthen route performance and bottom-line impact.

### 4.3 Funnel by Platform

To identify whether platform-specific issues may be contributing to user friction, we examine conversion performance across the transfer funnel for each platform.

Platform-level differences can often reveal UX inconsistencies, technical issues, or device-specific barriers that affect a user’s ability to complete a transfer.

In [11]:
# Execute SQL query
df = db.execute_sql_file("../sql/funnel_by_platform.sql")

# Plot funnel
fig = px.funnel(
    df,
    y="EVENT_NAME",
    x="USER_COUNT",
    color="PLATFORM",
    title="Conversion Funnel by Platform",
)
# Update traces
fig.update_traces(
    textinfo="value+percent previous",
    textposition="inside",
    hovertemplate="<b>%{y}</b><br>"
    + "Experience: <b>%{fullData.name}</b><br><br>"
    + "Users: <b>%{x:,}</b><br>"
    + "Conversion: <b>%{customdata[0]:.1%}</b><br>"
    + "Drop-off: <b>%{customdata[1]:.1%}</b><br>"
    + "Cumulative: <b>%{customdata[2]:.1%}</b><br>"
    + "<extra></extra>",
    customdata=df[
        ["CONVERSION_RATE", "DROP_OFF_RATE", "CUMULATIVE_CONVERSION_RATE"]
    ].values,
)

# Update layout
fig.update_layout(
    width=1200,
    height=500,
    yaxis_title_text="Funnel Stage",
    legend_title_text="Platform",
    template="plotly_white",
)

fig.show()

2025-12-09 22:18:00.854 | INFO     | src.database_manager:execute_sql_file:112 - Executing SQL file: ../sql/funnel_by_platform.sql
2025-12-09 22:18:00.866 | DEBUG    | src.database_manager:execute_query:88 - Query executed successfully. Returned 9 rows.


**Observations:**

- At this aggregate level, conversion rates appear fairly similar across platforms, with each showing roughly ~50% progression between stages. This uniformity suggests that high-level platform comparisons do not reveal meaningful differences or clear friction points.

### 4.4 Detailed Funnel: Platform × Region × Experience

Having examined funnel performance separately by region and by user experience, we drill down further to analyse platform × region × experience combinations.

This level of segmentation helps uncover highly specific friction patterns that may not be visible at broader levels of aggregation.

By isolating intersections such as Android users in APAC or new users on iOS in Europe, we can identify whether particular platform–market–experience combinations are driving disproportionate drop-off and contributing to overall funnel inefficiencies.

In [12]:
# Execute SQL query
df = db.execute_sql_file("../sql/funnel_by_all.sql")

# Get the list of default colors
default_colors = plotly.colors.qualitative.Plotly
unique_experiences = sorted(df["EXPERIENCE"].unique())
color_map = {
    exp: default_colors[i % len(default_colors)]
    for i, exp in enumerate(unique_experiences)
}

platforms = sorted(df["PLATFORM"].unique())
regions = sorted(df["REGION"].unique())

fig = make_subplots(
    rows=len(platforms),
    cols=len(regions),
    row_titles=list(platforms),
    column_titles=list(regions),
    shared_yaxes=True,
    vertical_spacing=0.05,
)

for r, platform in enumerate(platforms, start=1):
    for c, region in enumerate(regions, start=1):
        cell_df = df[(df["PLATFORM"] == platform) & (df["REGION"] == region)]

        for experience in sorted(cell_df["EXPERIENCE"].unique()):
            exp_data = cell_df[cell_df["EXPERIENCE"] == experience]

            fig.add_trace(
                go.Funnel(
                    name=experience,
                    y=exp_data["EVENT_NAME"],
                    x=exp_data["USER_COUNT"],
                    textinfo="value+percent previous",
                    textposition="inside",
                    marker={"color": color_map[experience]},
                    showlegend=(r == 1 and c == 1),
                ),
                row=r,
                col=c,
            )

fig.update_layout(
    title_text="Funnel Analysis: Platform × Region × Experience",
    width=1400,
    height=800,
    template="plotly_white",
    funnelmode="stack",
    legend_title_text="User Experience",
)

fig.show()

2025-12-09 22:18:00.896 | INFO     | src.database_manager:execute_sql_file:112 - Executing SQL file: ../sql/funnel_by_all.sql
2025-12-09 22:18:00.910 | DEBUG    | src.database_manager:execute_query:88 - Query executed successfully. Returned 54 rows.


**Observations:**
- The most critical friction point across the entire dataset is New APAC Android users.
    - Only ~1.6% complete a transfer end-to-end — the lowest conversion rate of any segment.
    - The steepest drop occurs at Transfer Created → Transfer Funded, with conversion of just ~1.8%, compared with 22% on Web and 31% on iOS for APAC new users.
    - This segment is the primary driver of APAC’s underperformance, making it a P0 priority for mobile and product teams.

- New APAC Web users also show substantial friction, particularly at the funding stage:
    - There is an ~80% drop-off from creation to funding.
    - This points to potential payment flow UX issues or integration limitations affecting new users on Web in APAC.
    
- North America shows a distinct platform pattern:
    - New Web users convert at just 34% from Created → Funded, significantly below Android and iOS.
    - This suggests region-specific funding or UX friction, potentially tied to local payment availability or partner bank behaviour.
    
- Existing users convert better than new users across all markets, but APAC continues to lag:
    - APAC existing users consistently show lower conversion (<70%) compared with Europe and North America.
    - In North America, existing Web users also underperform, which may indicate Web-specific UX friction or region-specific banking constraints.

- Europe has the most stable and balanced funnel across segments:
    - New users still underperform existing users, but the gap is smaller and no segment exhibits extreme friction.
    - Europe’s healthier conversion reinforces that the major issues are highly concentrated in specific platform–region combinations, rather than systemic across the entire route.


## 5. Friction Analysis (Time-to-Completion)

Beyond conversion rates, we also analyse the time users take to move between funnel stages (among those who successfully converted). Long delays can signal friction even among users who eventually complete their transfers, such as waiting for verification, unclear instructions, or slow funding methods.

To measure this, we identify the earliest event date for each user at each funnel stage and compute the time interval (in days) between consecutive steps. This allows us to understand how quickly users progress through the journey and to pinpoint stages where delays are most pronounced.

### 5.1 Average Friction by Region

Here, we plot the average friction (in days) across funnel stages by region to identify whether certain markets exhibit systematically longer delays or distinctive time-to-completion patterns.

In [13]:
# Execute SQL query
df = db.execute_sql_file("../sql/friction_duration.sql")

# Group by event and region
df = (
    df.groupby(["EVENT_NAME", "REGION"])
    .agg(AVG_FRICTION=("FRICTION", "mean"))
    .reset_index()
)

# Plot bar chart
fig = px.bar(
    df,
    x="REGION",
    y="AVG_FRICTION",
    color="EVENT_NAME",
    barmode="group",
    title="Average Days Between Transfer Stages by Region",
    template="plotly_white",
)
fig.add_hline(
    y=df["AVG_FRICTION"].mean(),
    line_dash="dash",
    line_color="red",
    annotation_text=f"Average: {df['AVG_FRICTION'].mean():.2f}",
)
fig.update_layout(
    width=1200,
    height=500,
    xaxis_title="Region",
    yaxis_title="No. of Days",
    legend_title_text="Stage Transition",
)
fig.show()

2025-12-09 22:18:00.954 | INFO     | src.database_manager:execute_sql_file:112 - Executing SQL file: ../sql/friction_duration.sql
2025-12-09 22:18:01.000 | DEBUG    | src.database_manager:execute_query:88 - Query executed successfully. Returned 29430 rows.


**Observations:**
- Europe exhibits the longest delay between Transfer Created → Transfer Funded, despite having relatively healthy conversion rates. This suggests that European users who do complete the step experience slower funding processes—potentially due to banking partner processing times, payment method delays, or compliance checks.

- APAC shows the shortest completion times across both transitions. This indicates that, for users who successfully progress through the funnel, the underlying payment and funding infrastructure operates efficiently, with minimal waiting time.

- North America sits between APAC and Europe, with moderate delays at both stages. This may reflect mixed dependencies across banks and funding methods but does not show any extreme friction at a regional level.

### 5.2 Friction by Platform × Region × Experience

Lastly, we perform a detailed breakdown—mirroring the earlier Platform × Region × Experience funnel analysis—to identify which specific segment combinations contribute most to time-to-completion delays.

This helps pinpoint targeted opportunities to improve the overall user journey for customers who ultimately complete their transfers.

In [14]:
# Execute SQL query
df = db.execute_sql_file("../sql/friction_duration.sql")

# Group by event, platform, region, experience
df = (
    df.groupby(["EVENT_NAME", "PLATFORM", "REGION", "EXPERIENCE"])
    .agg(AVG_FRICTION=("FRICTION", "mean"))
    .reset_index()
)

# Get unique values
event_names = sorted(df["EVENT_NAME"].unique())
num_regions = len(df["REGION"].unique())

# Calculate average friction by event
avg_by_event = df.groupby("EVENT_NAME")["AVG_FRICTION"].mean().to_dict()

# Plot bar chart
fig = px.bar(
    df,
    x="PLATFORM",
    y="AVG_FRICTION",
    color="EXPERIENCE",
    facet_col="REGION",
    facet_row="EVENT_NAME",
    barmode="group",
    title="Friction Analysis: Average Days by Platform × Region × Experience",
    template="plotly_white",
    labels={"AVG_FRICTION": "No. of Days"},
)

# Add mean line for each facet row
for i, event_name in enumerate(event_names, start=0):
    avg_value = avg_by_event[event_name]

    # Loop through each column in this row
    for col in range(1, num_regions + 1):
        fig.add_hline(
            y=avg_value,
            line_dash="dash",
            line_color="red",
            line_width=1.5,
            annotation_text=f"Mean: {avg_value:.2f} days"
            if col == num_regions // 3
            else "",
            annotation_position="top right",
            annotation={"xshift": 20},
            row=i,
            col=col,
        )

fig.update_layout(
    width=1500,
    height=800,
    xaxis_title="Region",
    legend_title_text="User Experience",
)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.show()

2025-12-09 22:18:01.038 | INFO     | src.database_manager:execute_sql_file:112 - Executing SQL file: ../sql/friction_duration.sql
2025-12-09 22:18:01.079 | DEBUG    | src.database_manager:execute_query:88 - Query executed successfully. Returned 29430 rows.


**Observations:**
- APAC Android New users exhibit the highest friction in the Created → Funded stage, averaging ~1.4 days, compared with only ~0.2 days on iOS—a 7× delay. This strongly indicates platform-specific issues in the Android funding or onboarding flow.

- New users in North America and Europe generally experience more friction in the early Created → Funded stage. However, in APAC, this pattern reverses: new iOS and web users progress relatively quickly, with Android being the sole outlier. This suggests that region-specific onboarding design may be required to reduce early-stage delays.

- Friction in the Funded → Transferred stage is driven primarily by existing users across regions. Since these users are already familiar with Wise, slowdowns here may point to operational delays, compliance checks, or payment settlement issues, rather than onboarding gaps.

## 6. Summary & Recommendations

### 6.1 Overall Assessment

The launch has validated strong underlying market demand, evidenced by high initial transfer creation volumes. However, sustaining growth has proven challenging. Early momentum tapered quickly, and critical conversion bottlenecks, particularly on Android in APAC, are preventing the route from reaching its projected business potential.

Without immediate product and UX interventions, the current trajectory will fall short of expectations despite promising early signals.

### 6.2 Key Performance Insights

1. **Market Demand**  
    - Demand is clearly present. Transfer creation trends confirm substantial latent interest in the product, showing that top-of-funnel awareness and acquisition are not the primary constraints.

2. **Growth Trends**  
    - Although the first two weeks saw a dramatic +283% week-over-week surge, growth quickly flattened and turned negative by Week 8.
    - The initial spike appears to have been launch-driven rather than indicative of sustained growth mechanisms.
    - This suggests the experience is not yet self-reinforcing, likely due to downstream friction.

3. **End-to-End Conversion**  
    - End-to-end conversion sits at ~24%, with a substantial drop-off between “Created” and “Transferred.”
    - This is a major limiter on throughput and ROI. Without addressing funnel leakage, additional demand generation efforts will yield diminishing returns.

4. **Regional Performance**  
    - APAC is emerging as the strongest-performing region, showing the highest growth trajectory.
    - This validates APAC as the primary market opportunity, yet also heightens the urgency to fix issues that disproportionately impact this segment (e.g., Android conversion).

5. **Platform Parity**  
    - A pronounced 60%+ conversion gap on Android represents the most critical systemic failure. Android users, particularly in APAC, are converting at extremely low rates, with signs pointing to UX and payment flow friction as the central cause.

### 6.3 Strategic Recommendations

1. **P0 — Fix Android APAC Conversion (Critical Path)**
    - Android APAC’s 1.6% end-to-end conversion, compared to 31% on iOS, represents the single largest drag on performance.
    - A deep examination of the Android funding flow, supported by user research, is essential. Addressing friction and improving clarity in the payment experience could unlock a substantial recovery in monthly transfers—potentially approaching iOS-level conversion (~25%).
    - **Why this matters:** APAC is the strongest growth region, and Android is the dominant platform. Fixing this one issue could meaningfully change overall business trajectory.

2. **P1 — Improve New User Activation**
    - New users convert at 18%, far below the 32% seen for existing users. The majority of drop-off occurs during funding, where unfamiliarity and documentation complexity create barriers.
    - **Recommendation:** Simplify onboarding, use progressive disclosure for compliance steps, and provide clearer guidance on funding methods. Improving new-user clarity will enhance acquisition efficiency, reduce early churn, and build a more repeatable growth engine.

3. **P2 — Reduce European Completion Times**
    - European transfers currently average 2.5 days, double the APAC timeline, largely due to banking and compliance delays.
    - **Recommendation:** Engage partners to tighten SLAs and introduce faster payment options where possible. Shorter completion times will lift user satisfaction (NPS) and strengthen repeat behavior, supporting long-term retention and revenue.